# Analyzing borrowers’ risk of defaulting




**The following analysis is devided into 3 steps:**

-processing of the data.

-categorization of the data.

-presenting the hypotheses results.



## General overview of the infornamtion - 

In [1]:
# Loading all the libraries
import pandas as pd 
import nltk
from nltk.stem import WordNetLemmatizer 
wordnet_lemma = WordNetLemmatizer()
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
# Load the data

credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [2]:
# Let's see how many rows and columns our dataset has

credit_scoring.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,19351.000000,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,26787.568355,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,16475.450632,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16488.504500,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32549.611000,NaN


In [3]:
# let's print the first N rows 

credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [4]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**comments**

- There appear to be duplicated valued in the [purpose] column that needs rewriting.

- There are big negative values in the [days_employed], it could mean that column is corrupted. The data needs to be rearranged and undergo abd() method.

- There could be duplicated values, we can locate them by the [total income] column.

In [5]:
#credit_scoring[credit_scoring[''].isnull()].count()
print(credit_scoring.isnull().sum())
print('')



print('Only columns with missing values:------------')

for i in credit_scoring:
    if credit_scoring[i].isnull().sum()>0:
        print(i)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Only columns with missing values:------------
days_employed
total_income



- there are missing values in colums 'days_employed' and 'total_income'. these colums are also the only float type colums in the data and they have the same number of missing values, which is interesting.

- we are looking at missing to check whether theres a pattern in the columns across the data. 

In [6]:


missing_days_employed = credit_scoring[credit_scoring.days_employed.isna()]
missing_total_income = credit_scoring[credit_scoring.total_income.isna()]

print('days employed NaN values - filtered table----------------------------------------------------------------------------')
print('')
print(missing_days_employed)
print('-----------------------------------------------------------------------------------------')
print(missing_days_employed.isnull()/len(missing_days_employed))
print('-----------------------------------------------------------------------------------------')
print('total income NaN values -  filtered table---------------------------------------------------------------------------')
print('')
print(missing_total_income)
print('-----------------------------------------------------------------------------------------')
print(missing_total_income.isnull()/len(missing_total_income))
print('-----------------------------------------------------------------------------------------')
print('days employed and total income in relation to family_status--------------------------------------------------------')
print(credit_scoring[credit_scoring.days_employed.isnull()]['family_status'].value_counts())
print('')
print(credit_scoring[credit_scoring.total_income.isnull()]['family_status'].value_counts())
print('-----------------------------------------------------------------------------------------')
print('days employed and total income in relation to children --------------------------------------------------------------')
print(credit_scoring[credit_scoring.total_income.isnull()]['children'].value_counts())
print('')
print(credit_scoring[credit_scoring.days_employed.isnull()]['children'].value_counts())







days employed NaN values - filtered table----------------------------------------------------------------------------

       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
2



-The 2 columns with missing values appear to be symmetric, both missing values in the same rows, by observing the size of NaN values across other columns.
we checked 2 different columns with value_counts method and found the same results for the given column with the missing values.

- we are now filtering the remaining columns from duplicated values since there are no more missin values.

In [7]:

column = ['children','family_status','income_type','days_employed','purpose','dob_years']

missing_values = []

for value in column:
    filtered_data = credit_scoring[credit_scoring['total_income'].isna()][value].value_counts()
    missing_values.append(filtered_data)
    print('missing values in ',value)
    print('-----------------')
    print(filtered_data)
    print('')
    
   

missing values in  children
-----------------
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

missing values in  family_status
-----------------
married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

missing values in  income_type
-----------------
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

missing values in  days_employed
-----------------
Series([], Name: days_employed, dtype: int64)

missing values in  purpose
-----------------
having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                           

**Intermediate conclusion**


- There are several conclutions we can make from the filtered data:
  
  The columns [days_employed] and [total_income] appear to have missing values in the same columns throught the data.
  It could suggest that a systematic error accured when data was generated. 
  
  The majority of rows with missing data appear to be valued as married employees with 0 kids.
  
  

-lets check the percentage of missing values in the columns of the data table to have the missing values in the same rows

In [8]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values

#print('percentage of missing values compared to the whole data set')
#print('days employed-----')
#print(len(credit_scoring[credit_scoring['days_employed'].isnull()])/len(credit_scoring) * 100)
#print('total income------')
#print(len(credit_scoring[credit_scoring['total_income'].isnull()])/len(credit_scoring) * 100)
#print('------------------')
print('patcentage of missing values within columns:------------')
print('')
print(credit_scoring.isnull().sum()/len(credit_scoring))




patcentage of missing values within columns:------------

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


In [9]:
# applying values to total_income by the mean of income type values.
credit_scoring[credit_scoring.total_income.isnull()]['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [10]:
credit_scoring['total_income_median']=credit_scoring['total_income']


In [11]:
credit_scoring['total_income_median'] = credit_scoring['total_income_median'].fillna(credit_scoring.groupby('income_type')['total_income'].transform('median'))


In [12]:
import pandas as pd 
credit_scoring[credit_scoring.total_income_median.isnull()]['income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

In [13]:
#Checking distribution
credit_scoring['total_income_median'].value_counts().describe(include='all')

count    19350.000000
mean         1.112403
std          9.286179
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max       1105.000000
Name: total_income_median, dtype: float64

In [14]:
credit_scoring[credit_scoring.total_income_median.isna()]['income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

In [15]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21525 non-null  int64  
 1   days_employed        19351 non-null  float64
 2   dob_years            21525 non-null  int64  
 3   education            21525 non-null  object 
 4   education_id         21525 non-null  int64  
 5   family_status        21525 non-null  object 
 6   family_status_id     21525 non-null  int64  
 7   gender               21525 non-null  object 
 8   income_type          21525 non-null  object 
 9   debt                 21525 non-null  int64  
 10  total_income         19351 non-null  float64
 11  purpose              21525 non-null  object 
 12  total_income_median  21525 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


**The column [income_type_mean] is our new [income_type] column as there are no missing values in the income_type_mean**





In [16]:
# Checking the distribution in the whole dataset

credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,total_income_median
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000,21525.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355,26436.051922
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632,15686.986477
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500,17247.708000
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000,22815.103500
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000,31287.991000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000,362496.645000


In [17]:
# Check for other reasons and patterns that could lead to missing values

credit_scoring[credit_scoring.duplicated('days_employed')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_median
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,24071.6695
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,18962.3180
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,24071.6695
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,18962.3180
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate,27577.2720
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car,27577.2720
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony,22815.1035
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property,27577.2720
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate,22815.1035


**Intermediate conclusion**

[Can we finally confirm that missing values are accidental? Check for anything else that you think might be important here.]

In [18]:
# Checking for other patterns - explain which

## Data transformation



In [19]:
# Let's see all values in education column to check if and what spellings will need to be fixed
credit_scoring['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

In [20]:
# Fix the registers if required
credit_scoring['education'] = credit_scoring.education.astype(str).str.lower()

In [21]:
# Checking all the values in the column to make sure we fixed them
credit_scoring['education'].value_counts()


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [22]:
# Let's see the distribution of values in the `children` column
credit_scoring['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [23]:
# [fix the data based on your decision]

credit_scoring['children'] = credit_scoring['children'].replace(-1,0)
credit_scoring['children'] = credit_scoring['children'].replace(20,2)


In [24]:
#  **I removed the value of '-1' from the column 'children' and replaced all its data to '0' instead**

#  **I also believe the value of '20' to be a typo of the value 2, so i replaced it across the data**

In [25]:
# Checking the `children` column again to make sure it's all fixed

credit_scoring['children'].value_counts()

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [26]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
credit_scoring['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

In [27]:
# Address the problematic values, if they exist

credit_scoring['days_employed_positive'] = credit_scoring['days_employed'].abs()
credit_scoring['days_employed_positive'] = credit_scoring['days_employed_positive'] / 24

credit_scoring['days_employed_positive'] = credit_scoring.groupby(['income_type', 'dob_years'])['days_employed_positive']\
    .transform(lambda x: x.fillna(x.mean()))

credit_scoring['days_employed_positive']= credit_scoring['days_employed_positive'].fillna(credit_scoring['days_employed_positive'].median())
credit_scoring['days_employed_positive']

0          351.569709
1          167.700156
2          234.309275
3          171.864467
4        14177.753002
             ...     
21520      188.721528
21521    14330.725172
21522       88.056120
21523      129.686738
21524       82.687816
Name: days_employed_positive, Length: 21525, dtype: float64

In [28]:
#   After filtering the values of days_employed column, by groupby method, and regarding the values as hours of being employes and not days.
#   It doesnt appers curropted and we can rely on its values.

In [29]:
# Check the result - make sure it's fixed
credit_scoring['days_employed_positive']


0          351.569709
1          167.700156
2          234.309275
3          171.864467
4        14177.753002
             ...     
21520      188.721528
21521    14330.725172
21522       88.056120
21523      129.686738
21524       82.687816
Name: days_employed_positive, Length: 21525, dtype: float64

**-The column [days_employed_positive] is our new [days_employed] column as there are no missing values in the days_employed_positive**

**-After preccesing the data it appears more organized, and perhaps can be used in our analysis if needed.**

In [30]:
# Check the `dob_years` for suspicious values and count the percentage

credit_scoring['dob_years'].value_counts()


35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [31]:
credit_scoring['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

[I changed the value 0 with the mean of the column.]

In [32]:
# Address the issues in the `dob_years` column, if they exist

credit_scoring['dob_years'] = credit_scoring['dob_years'].replace(0,43)

In [33]:
# Check the result - make sure it's fixed
credit_scoring['dob_years'].value_counts(ascending=True)

75      1
74      6
73      8
19     14
72     33
20     51
71     58
70     65
69     85
68     99
21    111
67    167
22    183
66    183
65    194
23    254
24    264
64    265
63    269
62    352
61    355
25    357
60    377
26    408
55    443
59    444
51    448
53    459
57    460
58    461
46    475
54    479
47    480
52    484
56    487
27    493
45    497
28    503
49    508
32    510
50    514
37    537
48    538
30    540
29    545
44    547
36    555
31    560
39    573
33    581
42    597
38    598
34    603
41    607
40    609
43    614
35    617
Name: dob_years, dtype: int64

In [34]:
# Let's see the values for the column
credit_scoring['family_status'].value_counts()



married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [35]:
# Address the problematic values in `family_status`, if they exist
credit_scoring['family_status'].value_counts()


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [36]:
# Check the result - make sure it's fixed


In [37]:
# Address the problematic values in `gender`, if they exist
credit_scoring['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [38]:
# lets see the values of purpose column
credit_scoring['purpose'].value_counts()

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
housing transactions                        653
buying property for renting out             653
transactions with commercial real estate    651
housing                                     647
purchase of the house                       647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

In [39]:
credit_scoring[credit_scoring.purpose.notna()]['purpose'].nunique()


38

In [40]:
credit_scoring[credit_scoring.purpose.notna()]['purpose'].unique()


array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [41]:
credit_scoring['education'] = credit_scoring['education'].str.lower()

In [42]:
wedding = ['wedding']
real_estate = ['house','housing','estate','renting','propery']
car = ['car','cars']
education =['education','educated','university'] 

In [43]:
m = Mystem()

In [44]:
def purpose_category(i):
    lemmatized = m.lemmatize(i)
    if any(word in lemmatized for word in real_estate):
        return 'real estate'
    elif any(word in lemmatized for word in wedding):
        return 'wedding'
    elif any(word in lemmatized for word in car):
        return 'car'
    elif any(word in lemmatized for word in education):
        return 'education'
    else:
        return 'other'
    


In [45]:
# Check the result - make sure it's fixed
credit_scoring['purpose_index'] = credit_scoring['purpose'].apply(purpose_category)

In [46]:
credit_scoring['purpose_index'].value_counts()

real estate    8951
car            4315
education      4022
wedding        2348
other          1889
Name: purpose_index, dtype: int64

In [47]:
# Let's see the values in the column
credit_scoring['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [48]:
# Address the problematic values, if they exist
credit_scoring['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [49]:
# Check the result - make sure it's fixed


In [50]:
# Checking duplicates
credit_scoring[credit_scoring.duplicated()] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_median,days_employed_positive,purpose_index
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family,22815.1035,99.744643,real estate
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,18962.3180,15169.758660,wedding
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony,22815.1035,83.443335,wedding
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony,18962.3180,15176.387882,wedding
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,18962.3180,15169.758660,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education,18962.3180,15221.286268,education
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated,18962.3180,15176.387882,education
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation,22815.1035,117.365059,real estate
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate,22815.1035,68.531248,real estate


In [51]:
# Address the duplicates, if they exist
credit_scoring[credit_scoring.duplicated()].sum()

children                                                                 16
days_employed                                                           0.0
dob_years                                                              3549
education                 secondary educationsecondary educationbachelor...
education_id                                                             61
family_status             marriedcivil partnershipcivil partnershipcivil...
family_status_id                                                         40
gender                    FFFFFMFFFFFFFFFFFFFFMFFFFFFFFFFFMMFFFFFFFFMMFM...
income_type               employeeretireeemployeeretireeretireeemployeer...
debt                                                                      0
total_income                                                            0.0
purpose                   purchase of the house for my familyto have a w...
total_income_median                                            1551695.4515
days_employe

In [52]:
# Last check whether we have any duplicates
credit_scoring.drop_duplicates(inplace=True)

In [53]:
del credit_scoring['total_income']

del credit_scoring['days_employed']

credit_scoring['total_income_median']=credit_scoring['total_income_median'].astype('int64')
credit_scoring['days_employed_positive']=credit_scoring['days_employed_positive'].astype('int64')

credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   children                21454 non-null  int64 
 1   dob_years               21454 non-null  int64 
 2   education               21454 non-null  object
 3   education_id            21454 non-null  int64 
 4   family_status           21454 non-null  object
 5   family_status_id        21454 non-null  int64 
 6   gender                  21454 non-null  object
 7   income_type             21454 non-null  object
 8   debt                    21454 non-null  int64 
 9   purpose                 21454 non-null  object
 10  total_income_median     21454 non-null  int64 
 11  days_employed_positive  21454 non-null  int64 
 12  purpose_index           21454 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.3+ MB


 **conclutions**
 
 **we chose to restore the missing values of the table with the mean of the column that the missing values were, with regards to its relevant values**
 
 We add new columns to the dataset. each new column contains filtered data of a previous lacking column.
 
 Since we removed all the problematic data from our data frame, thres no more duplicated no missing values.
 
 Now we can use our new filtered data for categorization and deeper analysis of the data.


## Categorization of data

[To answer the questions and test the hypotheses, you will want to work with categorized data. Look at the questions that were posed to you and that you should answer. Think about which of the data will need to be categorized to answer these questions. Below you will find a template through which you can work your way when categorizing data. The first step-by-step processing covers the text data; the second one addresses the numerical data that needs to be categorized. You can use both or none of the suggested instructions - it's up to you.]



In the following section we intend to categorize the data and to answer the following questions:

**heres a table oif all the relevant columns for our research questions:**


Is there a connection between having kids and repaying a loan on time?

Is there a connection between marital status and repaying a loan on time?

Is there a connection between income level and repaying a loan on time?

How do different loan purposes affect on-time loan repayment?

**in order to so do, we create a data table out of the columns we need for the categorization regarding the question above.** 



In [54]:
# Print the values for your selected data for categorization


credit_scoring_loan=credit_scoring.loc[:, credit_scoring.columns.drop(['education','family_status','gender','days_employed_positive','purpose'])]

credit_scoring_loan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   children             21454 non-null  int64 
 1   dob_years            21454 non-null  int64 
 2   education_id         21454 non-null  int64 
 3   family_status_id     21454 non-null  int64 
 4   income_type          21454 non-null  object
 5   debt                 21454 non-null  int64 
 6   total_income_median  21454 non-null  int64 
 7   purpose_index        21454 non-null  object
dtypes: int64(6), object(2)
memory usage: 1.5+ MB


In [55]:

def age_years(dob_years):
    return (2022 - dob_years)

credit_scoring_loan['age_years'] = credit_scoring_loan['dob_years'].apply(age_years) 

credit_scoring_loan[['age_years','dob_years']].value_counts()



age_years  dob_years
1987       35           616
1979       43           613
1982       40           607
1981       41           605
1988       34           601
1984       38           597
1980       42           596
1989       33           581
1983       39           572
1991       31           559
1986       36           554
1978       44           545
1993       29           544
1992       30           537
1974       48           536
1985       37           536
1972       50           513
1990       32           509
1973       49           508
1994       28           503
1977       45           496
1995       27           493
1970       52           484
1966       56           483
1975       47           477
1968       54           476
1976       46           472
1969       53           459
1965       57           456
1964       58           454
1971       51           446
1967       55           443
1963       59           443
1996       26           408
1962       60           374

[What main groups can you identify based on the unique values?]

Here we check whether having children has a relation to being in debt:

In [56]:
# Let's write a function to categorize the data based on common topics
debt_children = credit_scoring_loan.pivot_table(index='debt',values='children',aggfunc='sum')

print(debt_children)

      children
debt          
0         9318
1          945


The pivot table above shows that the relation between having children, by itself as a parameter, and not having children in relation to being in debt doesnt have an affect on .
 
 **Hence, we can presume that having children dont have an affect on return a loan.**

we can same the same about having or not education.
education status doesnt affect the ability to return a loan, as we see below.

In [57]:
debt_education = credit_scoring_loan.pivot_table(index='debt',values='education_id',aggfunc='sum')

debt_education

,education_id
debt,
0,15937
1,1593


In [58]:
 debt_family_status = credit_scoring_loan.pivot_table(index='debt',values='family_status_id',aggfunc='sum')

print(debt_family_status)

      family_status_id
debt                  
0                19029
1                 1865


Family status doestn have a relation to debt either. 

In [59]:
debt_income = credit_scoring_loan.pivot_table(index='debt',values='total_income_median')

print(debt_income)

      total_income_median
debt                     
0            26509.543499
1            25784.954049


## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [60]:
# Check the children data and paying back on time

credit_scoring_children = credit_scoring_loan[['children','debt']]

# Calculating default-rate based on the number of children

print(credit_scoring_children.value_counts())
print('')
print('pivot table:')
print(debt_children)

children  debt
0         0       13074
1         0        4364
2         0        1926
0         1        1064
1         1         444
3         0         303
2         1         202
4         0          37
3         1          27
5         0           9
4         1           4
dtype: int64

pivot table:
      children
debt          
0         9318
1          945


**Conclusion**

As we see in the column and the pivot table; There is no relation found between having children to not being able to return a loan.

**Is there a correlation between family status and paying back on time?**

In [61]:
# Check the family status data and paying back on time

credit_scoring_family_status = credit_scoring_loan[['family_status_id','debt']]


# Calculating default-rate based on family status

print(credit_scoring_family_status.value_counts())
print('')
print('pivot table:')
print(debt_family_status)

family_status_id  debt
0                 0       11408
1                 0        3763
4                 0        2536
3                 0        1110
0                 1         931
2                 0         896
1                 1         388
4                 1         274
3                 1          85
2                 1          63
dtype: int64

pivot table:
      family_status_id
debt                  
0                19029
1                 1865


Family status doesnt have a cerrelation between being unable to return a debt on time.

**Conclusion**

[Write your conclusions based on your manipulations and observations.]

**Is there a correlation between income level and paying back on time?**

we dont see a certain pattern.

In [62]:
# Check the income level data and paying back on time

credit_scoring_income = credit_scoring_loan[['total_income_median','debt']]


# Calculating default-rate based on income level

print(credit_scoring_income.sort_values('debt').value_counts(ascending=False))

print(credit_scoring_income.head(10))

total_income_median  debt
22815                0       973
27577                0       472
18962                0       353
24071                0       138
22815                1        98
                            ... 
20327                0         1
20328                0         1
20330                0         1
20340                0         1
362496               0         1
Length: 15948, dtype: int64
   total_income_median  debt
0                40620     0
1                17932     0
2                23341     0
3                42820     0
4                25378     0
5                40922     0
6                38484     0
7                21731     0
8                15337     0
9                23108     0



**Conclusion**

There is no distinct obresvation between total_income and debt.

**How does credit purpose affect the default rate?**

In [63]:
# Check the percentages for default rate for each credit purpose and analyze them

credit_scoring_purpose = credit_scoring_loan[['purpose_index','debt']]

print(credit_scoring_purpose.value_counts())


purpose_index  debt
real estate    0       8280
car            0       3903
education      0       3643
wedding        0       2138
other          0       1749
real estate    1        644
car            1        403
education      1        370
wedding        1        186
other          1        138
dtype: int64


**Conclusion**

We didnt find any relation between purpose of the loan, children number or martial status to having debt.

 

# General Conclusion 

After processig the credit scoring data; removing empty and duplicated values, rearranging new columns and and cetegorizing the data for our needs and hypotheses. We come to the conclutions in terms of relation to debt and the inability to return a loan:

-There is no distinct relation between having children and being unable to return a loan

-There is no distinct relation between any matrial status and being unable to return a loan

-There is no distinct relation between purpose of applying for a loan and being unable to return a loan
 
And There is no distinct relation between income level and being unable to return a loan.




